In [34]:
!pip install pmdarima
!pip install tensorflow

In [35]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
import itertools
import pmdarima
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout

In [36]:
#データの読み込み
df = pd.read_csv('stock_price.csv',parse_dates=True)
df=df.iloc[::-1]
df.reset_index(inplace=True,drop=True)
df

,日付け,終値,始値,高値,安値,出来高,変化率 %
0,1987-02-12,182.4,182.4,182.4,182.4,1.28B,16.25
1,1987-02-13,172.5,176.5,176.5,172.5,422.38M,-5.43
2,1987-02-16,175.5,166.7,175.5,165.7,360.51M,1.74
3,1987-02-17,182.4,176.5,184.3,175.5,881.51M,3.93
4,1987-02-18,191.2,186.3,191.2,186.3,795.09M,4.82
...,...,...,...,...,...,...,...
9197,2024-07-26,157.9,159.3,159.6,157.9,155.08M,-0.13
9198,2024-07-29,159.7,158.7,160.2,158.4,126.28M,1.14
9199,2024-07-30,158.7,158.8,159.2,158.0,138.14M,-0.63
9200,2024-07-31,160.4,158.2,160.7,158.1,173.91M,1.07


In [37]:
df_shift=df.shift(-1)
df["変化"]=df["終値"]-df_shift["終値"]
df

,日付け,終値,始値,高値,安値,出来高,変化率 %,変化
0,1987-02-12,182.4,182.4,182.4,182.4,1.28B,16.25,9.9
1,1987-02-13,172.5,176.5,176.5,172.5,422.38M,-5.43,-3.0
2,1987-02-16,175.5,166.7,175.5,165.7,360.51M,1.74,-6.9
3,1987-02-17,182.4,176.5,184.3,175.5,881.51M,3.93,-8.8
4,1987-02-18,191.2,186.3,191.2,186.3,795.09M,4.82,-4.9
...,...,...,...,...,...,...,...,...
9197,2024-07-26,157.9,159.3,159.6,157.9,155.08M,-0.13,-1.8
9198,2024-07-29,159.7,158.7,160.2,158.4,126.28M,1.14,1.0
9199,2024-07-30,158.7,158.8,159.2,158.0,138.14M,-0.63,-1.7
9200,2024-07-31,160.4,158.2,160.7,158.1,173.91M,1.07,4.1


In [38]:
df=df[["日付け","終値","始値","高値","安値","変化"]]
df.set_index(keys="日付け",inplace=True)
df=df[:-1]
df

,終値,始値,高値,安値,変化
日付け,,,,,
1987-02-12,182.4,182.4,182.4,182.4,9.9
1987-02-13,172.5,176.5,176.5,172.5,-3.0
1987-02-16,175.5,166.7,175.5,165.7,-6.9
1987-02-17,182.4,176.5,184.3,175.5,-8.8
1987-02-18,191.2,186.3,191.2,186.3,-4.9
...,...,...,...,...,...
2024-07-25,158.1,157.0,159.0,156.8,0.2
2024-07-26,157.9,159.3,159.6,157.9,-1.8
2024-07-29,159.7,158.7,160.2,158.4,1.0


In [39]:
df["増加"]=0
df["増加"][df["変化"]>0]=1
df

<ipython-input-39-67fc6d720a91>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["増加"][df["変化"]>0]=1
<ipython-input-39-67fc6d720a91>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

,終値,始値,高値,安値,変化,増加
日付け,,,,,,
1987-02-12,182.4,182.4,182.4,182.4,9.9,1
1987-02-13,172.5,176.5,176.5,172.5,-3.0,0
1987-02-16,175.5,166.7,175.5,165.7,-6.9,0
1987-02-17,182.4,176.5,184.3,175.5,-8.8,0
1987-02-18,191.2,186.3,191.2,186.3,-4.9,0
...,...,...,...,...,...,...
2024-07-25,158.1,157.0,159.0,156.8,0.2,1
2024-07-26,157.9,159.3,159.6,157.9,-1.8,0
2024-07-29,159.7,158.7,160.2,158.4,1.0,1


In [40]:
df_train=df[:int(len(df)*0.8)]
x_train=df_train[["終値","始値","高値","安値"]]
y_train=df_train[["増加"]]
df_test=df[int(len(df)*0.8):]
x_test=df_test[["終値","始値","高値","安値"]]
y_test=df_test[["増加"]]

In [41]:
def std_to_np(df):
    df_list = []
    df = np.array(df)
    for i in range(len(df)-40):
      df_s = df[i:i+40]
      scl = MinMaxScaler(feature_range=(0,1))
      df_std = scl.fit_transform(df_s)
      df_list.append(df_std)
    return np.array(df_list)

In [42]:
x_train_std=std_to_np(x_train)
x_test_std=std_to_np(x_test)

In [43]:
print(x_train_std.shape)
print(x_test_std.shape)

(7320, 40, 4)
(1801, 40, 4)


In [44]:
y_train=y_train[40:]
y_test=y_test[40:]
print(len(y_train))
print(len(y_test))

7320
1801


In [45]:
def lstm_comp(df):
  model=Sequential()
  model.add(LSTM(256, activation='relu', input_shape=(df.shape[1], df.shape[2])))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [46]:
model = lstm_comp(x_train_std)
result=model.fit(x_train_std,y_train,epochs=10,batch_size=64)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


115/115 ━━━━━━━━━━━━━━━━━━━━ 23s 178ms/step - accuracy: 0.5143 - loss: 0.6934
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 175ms/step - accuracy: 0.5030 - loss: 0.6934
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 19s 162ms/step - accuracy: 0.5090 - loss: 0.6929
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 165ms/step - accuracy: 0.5029 - loss: 0.6930
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 19s 156ms/step - accuracy: 0.5199 - loss: 0.6924
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 22s 170ms/step - accuracy: 0.5151 - loss: 0.6928
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 19s 159ms/step - accuracy: 0.5140 - loss: 0.6928
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.5065 - loss: 0.6931
Epoch 9/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.5193 - loss: 0.6925
Epoch 10/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.5147 - loss: 0.6928


In [47]:
pred=model.predict(x_test_std)

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step


In [48]:
#一致度
correct=0
incorrect=0
for i in range(len(pred)):
  if pred[i]>0.5 and y_test["増加"][i]==1:
    correct+=1
  elif pred[i]<0.5 and y_test["増加"][i]==0:
    correct+=1
  else:
    incorrect+=1
print("正解:"+str(correct))
print("不正解"+str(incorrect))
print("正解率"+str(correct/(correct+incorrect)))

正解:950
不正解851
正解率0.5274847307051638


<ipython-input-48-abd8342a3a5a>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif pred[i]<0.5 and y_test["増加"][i]==0:
